In [1]:
from riotwatcher import RiotWatcher,LolWatcher, ApiError
import pandas as pd
import numpy as np

In [2]:
#nombre invocador
user_name = 'the ghold liom  '
#region
region = 'la2'
#cola de traqueos ranked/flex/normal/aram
cola_raw = 'ranked'
# Usa tu clave de API aquí 
#ESTA API CAMBIA DIARIAMENTE, SE DEBE ACTUALIZAR
api_key = 'RGAPI-fc66b1b8-c5fc-4264-adc7-92a45f5f53cd'
#cantidad de partidas a buscar
num_partidas_input = 100



In [3]:
# Crea un objeto de LolWatcher
lol_watcher = LolWatcher(api_key)
riot_watcher = RiotWatcher(api_key)
# Constantes generales
NORMAL_RECL_ = 400;
RANKED_SOLO_ = 420;
NORMAL_BLIND_ = 430;
RANKED_FLEX_ = 440;
ARAM_ = 450;
DICC_COLA = {'normal':NORMAL_RECL_,
'ranked':RANKED_SOLO_ ,
'blind':NORMAL_BLIND_ ,
'flex':RANKED_FLEX_,
'aram':ARAM_ }
cola = DICC_COLA[cola_raw]

dicc_dtype2sql = {np.dtype('int64'):'INT',
                        np.dtype('float64'):'FLOAT',
                        np.dtype('O'):'VARCHAR(30)',
                        np.dtype('bool'):'BOOL'}

PUUID_USER = riot_watcher.account.by_riot_id(region='AMERICAS',game_name=user_name,tag_line='LAS')['puuid']

usuario = lol_watcher.summoner.by_puuid(region, PUUID_USER)
usuario

{'id': '1y6Sdjpmca2rr2aRTvjyBmyDI_3nF661HJEJUNVdwXCBPXY',
 'accountId': '7oBpsTgaj7get1KK-DwtOG8RK0KD6GNYesvdDeac5efQoFAwOcX44IdX',
 'puuid': '2Mtva55S5DzLXzACO_JXB5rc8a9fSJgX7nC_fw569bSF85F7BrYB-WFUiVV3Rnzxckd-aNjIlD39Gg',
 'profileIconId': 4902,
 'revisionDate': 1714240054830,
 'summonerLevel': 378}

In [4]:
import time
def get_last_ranked(usuario,region=region,queue=RANKED_SOLO_):
    return lol_watcher.match.matchlist_by_puuid(region, usuario["puuid"],count=2,queue=queue)[0]
def get_last_match(usuario,region=region):
    return lol_watcher.match.matchlist_by_puuid(region, usuario["puuid"],count=2)[0]
def get_previous_matches(summoner_puuid, region, match_id,queue=None,count=10):
    try:
        #ARREGLAR ESTA LINEA PARA QUE BUSQUE EL MATCHID
        previous_matches = []
        found_match = False
        idx_init=0
        search = min([count+20,100])
        keep = True
        while (not found_match) or keep:
            matchlist = lol_watcher.match.matchlist_by_puuid(region, summoner_puuid,count = search,start=idx_init,queue=queue)
            if len(matchlist) == 0:
                break
            elif keep:
                for match in matchlist:
                    if match == match_id:
                        found_match = True
                    elif found_match:
                        previous_matches.append(match)
                        if len(previous_matches) >= count:
                            keep = False
                            break
                idx_init += search
            
        return previous_matches
    except ApiError as err:
        print(f"Error retrieving previous matches: {err}")
        return None
def get_match_stats(matchid,region,include_teams = False):
    if isinstance(matchid,str):
        matchid = [matchid]
    elif isinstance(matchid,list):
        pass
    else:
        raise TypeError("Solo formato string o list son permitidos.")
    matches_dict = dict()
    i = 0
    for m_id in matchid:
        if len(matchid) > 100: 
            if (i+1)%50 == 0:
                print(f"Obteniedo estadisticas para {i+1} partidas\n")
            #time.sleep(1) 
        elif len(matchid) <= 100 and i%10 == 0:
            print(f"Obteniedo estadisticas para {i+1} partidas\n")
        #print información
        #print(f"Obteniedo estadisticas de la partida {m_id}\n")
        matches_dict[m_id] = lol_watcher.match.by_id(region, m_id)['info']['participants']
        if include_teams:
            print("incluyendo información de teams\n")
            matches_dict[m_id].append(lol_watcher.match.by_id(region, m_id)['info']['teams'])
        i += 1
    return matches_dict

        

In [8]:
from types import FunctionType
import time
import pymysql
import re

class playerData(object):
    def F00_match_stats(self):
        self.match_stats = get_match_stats(self._LMlist,region,include_teams=False) 
    def F01_dataframe(self):
        df_list = list()
        for mid,lista in self.match_stats.items():
            for dicc in lista:
                df = pd.DataFrame({x:[y] for x,y in dicc.items() if x not in ['challenges','missions','perks']})
                df['idpartida'] = mid
                df_list.append(df)
        print('Consolidando DataFrame...\n')
        df = pd.concat(df_list).fillna('').reset_index(drop=True)
        cols = [x for x in df.columns if x not in ['riotIdGameName','idpartida']]
        cols = ['idpartida','riotIdGameName'] + cols
        self.dataframe = df[cols]
        print('DataFrame generado.\n')
    def F02_createTable(self):
        schema = [x + ' ' + dicc_dtype2sql[y] + ' NOT NULL' for x,y in self.dataframe.dtypes.items() if x in ('riotIdGameName','idpartida')] +\
        [x + ' ' + dicc_dtype2sql[y] for x,y in self.dataframe.dtypes.items() if x not in ('riotIdGameName','idpartida')]              
        schema = ', '.join(['id INT AUTO_INCREMENT PRIMARY KEY']+schema)
        self.altercols = {'add':['riotIdGameName','summonerId','puuid'],
                          'encoding':['riotIdTagline','riotIdGameName','summonerName']}
        self.schema = schema
        query_create = f"CREATE TABLE IF NOT EXISTS partidas ({schema});"
        conn = pymysql.connect(host="localhost",user="root",passwd="12345",db="lol")
        with conn:
            with conn.cursor() as cursor:
                cursor.execute(query_create)
                conn.commit()
    def F03_checkcolumns(self):
        conn = pymysql.connect(host="localhost",user="root",passwd="12345",db="lol")
        with conn:
            with conn.cursor() as cursor:
                cols_query = "SHOW COLUMNS FROM partidas"
                cursor.execute(cols_query)
                cols_sql = [row[0] for row in cursor.fetchall()]
                cols_df = self.dataframe.columns.tolist()
                if len(cols_df) > len(cols_sql):
                    cols_extra = set([x for x in cols_df if x not in cols_sql])
                    self.altercols['addfields'] = list({x : dicc_dtype2sql[y]  for x,y in self.dataframe.dtypes.items()\
                                                        if x in cols_extra}.items())
    def F04_alterTable(self):
        querys = [f"ALTER TABLE partidas MODIFY {q} VARCHAR(100)"\
                     for x,y in self.altercols.items() if x == 'add' for q in y]
        querys += [f"ALTER TABLE partidas MODIFY COLUMN {q} VARCHAR(255) CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci NOT NULL;"\
                     for x,y in self.altercols.items() if x == 'encoding' for q in y]
        querys += [f"ALTER TABLE partidas ADD {q[0]} {q[1]}"\
                     for x,y in self.altercols.items() if x == 'addfields' for q in y]
        conn = pymysql.connect(host="localhost",user="root",passwd="12345",db="lol")
        with conn:
            with conn.cursor() as cursor:
                for qu in querys:
                    cursor.execute(qu)
                    conn.commit()
    def F05_uploadSQL(self):
        ssf2 = (len(self.dataframe.columns.tolist())-1)*'%s,' + '%s'
        ssf1 = ','.join(self.dataframe.columns.tolist())
        datos = [tuple(x) for x in self.dataframe.values]
        insert_query = f"INSERT INTO partidas ({ssf1}) VALUES ({ssf2});"
        conn = pymysql.connect(host="localhost",user="root",passwd="12345",db="lol")
        with conn:
            with conn.cursor() as cursor:
                cursor.executemany(insert_query,datos)
                conn.commit()
    def __run__(self):
        for x in dir(self):
            if 'F00' in x and self._nummatch >= 100: 
                print('Esperando 30 seg para requests de api...\n')
                time.sleep(30)   
            if x.startswith('F'):
                print(x)
                getattr(self,x)()                             
    def __init__(self,user_name,region):
        self._nummatch = num_partidas_input
        self._puuid = riot_watcher.account.by_riot_id(region='AMERICAS',game_name=user_name,tag_line='LAS')['puuid']
        self._usuario = lol_watcher.summoner.by_puuid(region, self._puuid)
        self._LMseed = get_last_ranked(self._usuario,region=region)
        self._LMlist = get_previous_matches(self._puuid, region=region,match_id=self._LMseed,queue=RANKED_SOLO_,count=self._nummatch)

In [9]:
usuario_data = playerData(user_name,region)

In [10]:
time.sleep(120)

In [11]:
usuario_data.__run__()
        

Esperando 30 seg para requests de api...

F00_match_stats
Obteniedo estadisticas para 1 partidas

Obteniedo estadisticas para 11 partidas

Obteniedo estadisticas para 21 partidas

Obteniedo estadisticas para 31 partidas

Obteniedo estadisticas para 41 partidas

Obteniedo estadisticas para 51 partidas

Obteniedo estadisticas para 61 partidas

Obteniedo estadisticas para 71 partidas

Obteniedo estadisticas para 81 partidas

Obteniedo estadisticas para 91 partidas

F01_dataframe
Consolidando DataFrame...

DataFrame generado.

F02_createTable
F03_checkcolumns
F04_alterTable
F05_uploadSQL


In [18]:
conn = pymysql.connect(host="localhost",user="root",passwd="12345",db="lol")
with conn:
    sql_query = "SELECT DISTINCT * FROM partidas"
    df = pd.read_sql_query(sql_query, conn)
    print(df)

C:\Users\ikeem\AppData\Local\Temp\ipykernel_13816\2195779161.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


        id       idpartida    riotIdGameName  allInPings  assistMePings  \
0        1  LA2_1406516876        JoooKeerRr           0              0   
1        2  LA2_1406516876          Zairorod           0              1   
2        3  LA2_1406516876       Poroteichon           0              0   
3        4  LA2_1406516876         Gato ateo           0              0   
4        5  LA2_1406516876   El Mas Toxico l           3              0   
...    ...             ...               ...         ...            ...   
1095  3096  LA2_1382995930         Azrael451           0              0   
1096  3097  LA2_1382995930   de2dehasbullita           0              7   
1097  3098  LA2_1382995930     capitannemo76           1              1   
1098  3099  LA2_1382995930   SirPedroCastell           0              0   
1099  3100  LA2_1382995930      SaKe D ßinKs           0              4   

      assists  baronKills  basicPings  bountyLevel  champExperience  ...  \
0           0          